In [1]:
import os
os.chdir("/home/ybang-eai/research/2024/ROMARL/ROMARL")
from TwoStageROProcessEnvironment.env.PressureControlledTwoStageROProcess_simple import TwoStageROProcessEnvironment
from algorithms.mixer.QMIX import QMixer, VDN, ReplayBuffer, PrioritizedExperienceReplay, RNNAgent, CentralizedRNNAgent, mask_and_softmax, softmax_and_mask, mask_and_nothing, centralized_mask_and_nothing
import atexit
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from copy import copy, deepcopy
from utils.epsilon_greedy import epsilon_greedy, EpsilonManager, boltzmann_policy, greedy_action_policy, get_action_from_q, centralized_epsilon_greedy, DistributedActionController, CentralizedDistributedActionController
from utils.descript import save_as_markdown
import itertools
from matplotlib import pyplot as plt
import pandas as pd
import os
from datetime import datetime
import questionary
import argparse
import shutil
from tqdm import tqdm
import pickle

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [ ]:
# Path in which the simulation data are stored. The data uploaded (HinderOneAll.zip) is compressed as a zip file, thus needs to be extracted.
exp_path = {
    "VDN":  "/home/ybang-eai/research/2024/ROMARL/ROMARL/interpretability/VDN/25.03.05.15.11"
}

# Path in which the trained parameters are stored. Only episode 49000's parameters are uploaded.
param_path = {
    "VDN":  "/home/ybang-eai/research/2024/ROMARL/ROMARL/parameters/VDN/"
}

exp_type = ["HinderOneAll"]

target_episode = "49000"

In [3]:
def load_model_parameters(agent_nets, directory, device):
    """
    Load the parameters of the agent networks from the specified directory.

    Args:
        agent_nets (dict): Dictionary of agent networks to which parameters will be loaded.
        directory (str): The directory from where the model parameters should be loaded.
    """
    # Load parameters for each agent network
    for agent_id, agent in agent_nets.items():
        agent_state_dict = torch.load(os.path.join(directory, f'agent_{agent_id}_params.pt'), map_location = device)
        agent.load_state_dict(agent_state_dict)

In [5]:
render_mode = 'silent'
save_dir = os.path.join('/home/ybang-eai/research/2024/ROMARL/ROMARL/interpretability', "VDN", datetime.now().strftime("%y.%m.%d.%H.%M"))

print("Environment setup ...")
env = TwoStageROProcessEnvironment(render_mode=render_mode, len_scenario=None, save_dir=save_dir)
print("Environment setup done.")

agents = env.agents
n_agents = len(agents)


parameter_algorithm = "VDN"

device = 'cuda:0'
agent_nets = {
    a: RNNAgent(input_shape=env.observation_space(a).shape[0], n_hidden_dim=64, n_actions=env.action_space(a).n).to(device) for a in agents
}
load_model_parameters(agent_nets, os.path.join(param_path[parameter_algorithm], target_episode), device)
files_in_path = os.listdir(os.path.join(exp_path[parameter_algorithm], exp_type[0]))
files_in_path = [f for f in files_in_path if f.endswith(".pkl")]


data_algorithm = 'VDN'

data = []
for file in files_in_path:
    split_elements = file.split('_')
    episode = split_elements[0]
    agent_hindered = split_elements[1]
    feed_concentration = split_elements[2]
    step = split_elements[-1].split('.')[0]
    parameter_type = '_'.join(split_elements[3:-1])
    data.append([episode, agent_hindered, feed_concentration, parameter_type, step, file])

df_files = pd.DataFrame(data, columns=["episode", "agent_hindered", "feed_concentration", "parameter_type", "step", "file_path"])

# Strip "ppm" and convert to float
df_files['feed_concentration'] = df_files['feed_concentration'].str.replace('ppm', '').astype(float)
df_files['step'] = df_files['step'].astype(int)

df_files = df_files.drop(columns=['episode', 'agent_hindered'])

# Group by feed_concentration
grouped_df = df_files.groupby('feed_concentration')
sorted_grouped_df = grouped_df.apply(lambda x: x.sort_values(by='step')).reset_index(drop=True)
df_files_sorted = df_files.sort_values(by=['feed_concentration', 'step']).reset_index(drop=True)
feed_concentrations = df_files_sorted['feed_concentration'].unique()
parameter_types     = df_files_sorted['parameter_type'].unique()
steps               = df_files_sorted['step'].unique()

input_by_concentration = {}
hidden_by_concentration = {}
agent_q_by_concentration = {}
for feed_c in feed_concentrations:
    input_by_concentration[feed_c] = []
    target_df = df_files.loc[(df_files['feed_concentration'] == feed_c) & (df_files['parameter_type'] == "previous_observations_scaled")].sort_values(by='step')
    target_files = target_df['file_path'].values
    for file in target_files:
        with open(os.path.join(exp_path[data_algorithm], exp_type[0], file), 'rb') as f:
            data = pickle.load(f)
            input_by_concentration[feed_c].append(data)
            
    target_hidden_df = df_files.loc[(df_files['feed_concentration'] == feed_c) & (df_files['parameter_type'] == "hiddens")].sort_values(by='step')
    target_hidden_files = target_hidden_df['file_path'].values
    hidden_by_concentration[feed_c] = []
    for file in target_hidden_files:
        with open(os.path.join(exp_path[data_algorithm], exp_type[0], file), 'rb') as f:
            data = pickle.load(f)
            hidden_by_concentration[feed_c].append(data)
    target_agent_q_df = df_files.loc[(df_files['feed_concentration'] == feed_c) & (df_files['parameter_type'] == "agent_qs")].sort_values(by='step')
    target_agent_q_files = target_agent_q_df['file_path'].values
    agent_q_by_concentration[feed_c] = []
    for file in target_agent_q_files:
        with open(os.path.join(exp_path[data_algorithm], exp_type[0], file), 'rb') as f:
            data = pickle.load(f)
            agent_q_by_concentration[feed_c].append(data)

Environment setup ...
Environment setup done.


/home/ybang-eai/miniconda3/envs/ROMARL/lib/python3.11/site-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/home/ybang-eai/miniconda3/envs/ROMARL/lib/python3.11/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(
/tmp/ipykernel_1277567/2799131360.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sorted_grouped_df = grouped_df.apply(lambda x: x.sort_values(by='step')).reset_index(drop=True)


In [7]:
input_by_concentration[300.0]

[{'influent_flowrate': {'observation': array([ 0.42857152,  1.1117495 , -0.07449962, -0.466077  , -0.3691713 ,
           0.5119443 ], dtype=float32),
   'action_mask': array([1, 1, 1, 1, 1], dtype=int8)},
  '1st_stage_pump': {'observation': array([ 0.5119443 ,  0.12574497,  0.42857152,  4.2754345 , -0.07449963,
           0.0712607 ,  0.96338856,  0.36428708,  0.4       ,  0.96704966],
         dtype=float32),
   'action_mask': array([1, 1, 1, 1, 1], dtype=int8)},
  '2nd_stage_pump': {'observation': array([ 0.5119443 ,  4.2754345 , -0.07449962,  0.36428708,  2.599462  ,
          -0.05296502,  0.03312572, -0.09603422,  0.3807748 ,  0.02      ,
           0.96704966], dtype=float32),
   'action_mask': array([1, 1, 1, 1, 1], dtype=int8)}},
 {'influent_flowrate': {'observation': array([ 0.44285718,  1.0989012 , -0.054823  , -0.38223884, -0.36446464,
           0.5012058 ], dtype=float32),
   'action_mask': array([1, 1, 1, 1, 1], dtype=int8)},
  '1st_stage_pump': {'observation': array([ 0

In [41]:
input_by_agent = {}
for agent in agents:
    input_by_agent[agent] = []
    for feed_c in feed_concentrations:
        segment_by_concentration = []
        for obs in input_by_concentration[feed_c]:
            segment_by_concentration.append(torch.from_numpy(obs[agent]['observation']).to(device).float().unsqueeze(0))
        input_by_agent[agent].append(segment_by_concentration)
        
hidden_by_agent = {}
for agent in agents:
    hidden_by_agent[agent] = []
    for feed_c in feed_concentrations:
        segment_by_concentration = []
        for hidden in hidden_by_concentration[feed_c]:
            segment_by_concentration.append((hidden[agent]).to(device).float())
        hidden_by_agent[agent].append(segment_by_concentration)

agent_q_by_agent = {}
for agent in agents:
    agent_q_by_agent[agent] = []
    for feed_c in feed_concentrations:
        segment_by_concentration = []
        for agent_q in agent_q_by_concentration[feed_c]:
            segment_by_concentration.append(agent_q[agent].to(device).float())
        agent_q_by_agent[agent].append(segment_by_concentration)


In [43]:
model_features = {
    'influent_flowrate': ['feed_flowrate', '1st_permeate_flowrate', '2nd_feed_flowrate', '2nd_permeate_flowrate', '2nd_brine_flowrate', 'temperature'],
    '1st_stage_pump':['temperature', 'feed_concentration', 'feed_flowrate', '1st_brine_concentration',
                    '1st_brine_flowrate', '1st_permeate_concentration', '1st_permeate_flowrate', '1st_brine_pressure',
                    '1st_pressure_applied', '1st_recovery'],
    '2nd_stage_pump':['temperature', '2nd_feed_concentration', '2nd_feed_flowrate', '1st_brine_pressure', '2nd_brine_concentration', '2nd_brine_flowrate',
                      '2nd_permeate_concentration', '2nd_permeate_flowrate', '2nd_brine_pressure', '2nd_pressure_applied', '1st_recovery']
                    }

input_by_agent_df = {}
for agent in agents:
    input_by_agent_df[agent] = []
    for feed_c in feed_concentrations:
        segment_by_concentration = []
        for obs in input_by_concentration[feed_c]:
            obs_numpy = (obs[agent]['observation'])
            df = pd.DataFrame(obs_numpy.reshape(1, -1), columns=model_features[agent])
            segment_by_concentration.append(df)
        # reset index of segment_by_concentration's resulting df before appending
        input_by_agent_df[agent].append(pd.concat(segment_by_concentration, ignore_index=True))

In [44]:
input_by_agent_df['1st_stage_pump'][0]

,temperature,feed_concentration,feed_flowrate,1st_brine_concentration,1st_brine_flowrate,1st_permeate_concentration,1st_permeate_flowrate,1st_brine_pressure,1st_pressure_applied,1st_recovery
0,0.505180,0.124724,0.428572,3.878402,-0.071178,0.065933,0.960067,0.363588,0.400,0.964060
1,0.507382,0.123205,0.442857,4.101611,-0.072976,0.067982,0.972976,0.368479,0.405,0.966018
2,0.498177,0.125857,0.428572,4.265010,-0.074444,0.069814,0.963333,0.373608,0.410,0.966999
3,0.509469,0.126795,0.414286,5.231101,-0.080932,0.090435,0.958710,0.380840,0.415,0.972564
4,0.495704,0.125119,0.400000,4.978820,-0.080007,0.081128,0.946673,0.380079,0.415,0.971435
...,...,...,...,...,...,...,...,...,...,...
114,0.497536,0.126711,0.407143,4.444689,-0.076610,0.059684,0.948832,0.451696,0.490,0.968476
115,0.505507,0.124535,0.392857,5.590530,-0.083613,0.069588,0.944724,0.453207,0.490,0.974617
116,0.509198,0.122532,0.400000,5.593691,-0.083634,0.069358,0.950301,0.453154,0.490,0.974766
117,0.496515,0.123755,0.407143,4.045702,-0.073765,0.056591,0.945987,0.451389,0.490,0.965876


In [45]:
batch_input = {}

for agent in agents:
    # Stack the tensors for the specified agent
    stacked_tensor = torch.stack([torch.cat(segment, dim=0) for segment in input_by_agent[agent]])

    batch_input[agent] = stacked_tensor

In [46]:
obs_hidden_by_agent = {}
for agent in agents:
    obs_hidden_by_agent[agent] = []
    for i, _ in enumerate(hidden_by_agent[agent]):
        obs_hidden_seg = [(obs, hidden.squeeze()) for obs, hidden in zip(batch_input[agent][i], hidden_by_agent[agent][i])]
        obs_hidden_by_agent[agent].append(obs_hidden_seg)

In [47]:
len(obs_hidden_by_agent['1st_stage_pump'][0])

119

In [50]:
import shap

def explain_rnn_agent_kernel(agent_net, obs_hidden_pairs):
    import shap
    """
    Generate SHAP explanations for an RNN agent using KernelExplainer.
    Expects obs_hidden_pairs to be a list of (observation, hidden) tuples.
    """
    agent_net.eval()
    
    all_observations = []
    all_hidden_states = []
    for obs_hidden_sequence in obs_hidden_pairs:
        for obs, hidden in obs_hidden_sequence:
            all_observations.append(obs)
            all_hidden_states.append(hidden)
            
    assert len(all_observations) == len(all_hidden_states), "Mismatch in observations and hidden states"

    observations = torch.stack(all_observations)
    hidden_states = torch.stack(all_hidden_states)
    
    # Optionally squeeze extra dimensions from hidden if necessary
    if len(hidden_states.shape) == 3 and hidden_states.shape[1] == 1:
        hidden_states = hidden_states.squeeze(1)
    
    # Concatenate along feature dimension
    combined_data = torch.cat([observations, hidden_states], dim=1)
    print(f"Combined data shape: {combined_data.shape}")  # Should be (n_samples, obs_dim+hidden_dim)
    
    # Convert to numpy and split into background and test
    combined_data_np = combined_data.detach().cpu().numpy()
    # Use shap.kmeans to extract 500 representative samples
    reduced_background = shap.kmeans(combined_data_np, 100)

    # The cluster centers are stored in the `.data` attribute:
    background_samples = reduced_background.data
    
    # Set obs_dim and hidden_dim for the prediction function
    global obs_dim, hidden_dim
    obs_dim = observations.shape[1]
    hidden_dim = hidden_states.shape[1]
    
    # Create a prediction function that wraps your agent network
    def predict_fn(X):
        X_tensor = torch.from_numpy(X).float().to(device)
        obs = X_tensor[:, :obs_dim]
        hidden = X_tensor[:, obs_dim:]
        with torch.no_grad():
            output, _ = agent_net(obs, hidden)
        return output.cpu().numpy()
    
    # Create KernelExplainer and compute SHAP values
    explainer = shap.KernelExplainer(predict_fn, background_samples)
    shap_values = explainer.shap_values(combined_data_np, nsamples=500)
    
    return shap_values, observations, hidden_states

In [51]:
for agent_name in agents:
    shap_values, observations, hidden_states = explain_rnn_agent_kernel(
        agent_nets[agent_name],
        obs_hidden_by_agent[agent_name]
    )

    # Save the SHAP values, observations and hidden_states to pickled files
    with open(f'shap_values_{agent_name}.pkl', 'wb') as f:
        pickle.dump(shap_values, f)

    with open(f'observations_{agent_name}.pkl', 'wb') as f:
        pickle.dump(observations, f)

    with open(f'hidden_states_{agent_name}.pkl', 'wb') as f:
        pickle.dump(hidden_states, f)

Combined data shape: torch.Size([11900, 70])


  0%|          | 0/11900 [00:00<?, ?it/s]

Combined data shape: torch.Size([11900, 74])


  0%|          | 0/11900 [00:00<?, ?it/s]

Combined data shape: torch.Size([11900, 75])


  0%|          | 0/11900 [00:00<?, ?it/s]